In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import wilcoxon

In [2]:
def get_price(start, end, ticker):
    """
    Fetch daily adjusted closing prices of a stock from Yahoo Finance.

    Parameters:
    Start (str): Start date for the price data.
    End (str): End date for the price data.
    Ticker (str): Ticker symbol of the stock.

    Returns:
    list: List of daily adjusted closing prices.
    """
    df = yf.download(ticker, start=start, end=end)
    return df['Adj Close'].tolist()

# Get actions for different models
def get_action(start, end, ticker, file_path, col):
    """
    Fetch action data from a specified CSV file.

    Parameters:
    start (str): Start date for the action data.
    end (str): End date for the action data.
    ticker (str): Ticker symbol of the stock (not used in the function).
    file_path (str): Path to the CSV file containing the action data.
    col (list): Column names to be used for date and action.

    Returns:
    list: List of actions corresponding to the specified date range.
    """
    df = pd.read_csv(file_path)
    df[col[0]] = pd.to_datetime(df[col[0]])
    action = df[(df[col[0]] >= pd.to_datetime(start)) & (df[col[0]] < pd.to_datetime(end))][col[1]].tolist()
    return action

def reward_list(price:list, actions:list):
    reward = 0
    reward_list = []
    for i in range(len(price)-1):
        reward += actions[i] * np.log(price[i+1]/price[i])
        reward_list.append(reward)
    return reward_list

def calculate_metrics(price, actions):
    """
    Calculate various financial metrics based on price and actions.

    Parameters:
    price (list): List of daily prices.
    actions (list): List of actions taken.

    Returns:
    tuple: A tuple containing calculated metrics
    """
    daily_rw = daily_reward(price, actions)
    cum_return = reward_list(price, actions)
    return daily_rw, cum_return

def perform_wilcoxon_test(spr1, spr2):
    stat, p = wilcoxon(spr1, spr2)
    return stat, p

In [4]:
if __name__ == "__main__":
    ticker = 'TSLA'
    start = '2022-06-25'
    end = '2023-04-25'

    # Fetch price data
    price = get_price(start, end, ticker)

    # Define file paths and columns for action data
    file_path_Me = '/Users/yuechenjiang/Desktop/CatMemo/result/Tsla-new-full.csv'
    file_path_Park = '/Users/yuechenjiang/Desktop/CatMemo/result/action_df_tsla_park_v2.csv'
    col_Me = ['date', 'direction']
    file_path_GPT = '/Users/yuechenjiang/Desktop/CatMemo/BenchMark/fingpt/tsla_curie.csv'
    col_GPT = ['dates', 'actions']
    file_path_A2C = '/Users/yuechenjiang/Desktop/CatMemo/result/TSLA_A2C_summary_data_seed2_full.csv'
    file_path_PPO = '/Users/yuechenjiang/Desktop/CatMemo/result/TSLA_PPO_summary_data_seed1_full.csv'
    file_path_DQN = '/Users/yuechenjiang/Desktop/CatMemo/result/TSLA_DQN_summary_data_seed1_full.csv'
    col_RL = ['date', 'action']

    # Fetch action data
    FinMe = get_action(start, end, ticker, file_path_Me, col_Me)
    Park = get_action(start, end, ticker, file_path_Park, col_Me)
    FinGPT = get_action(start, end, ticker, file_path_GPT, col_GPT)
    A2C = get_action(start, end, ticker, file_path_A2C, col_RL)
    PPO = get_action(start, end, ticker, file_path_PPO, col_RL)
    DQN = get_action(start, end, ticker, file_path_DQN, col_RL)
    
    # Calculate rewards
    FinMe_rw = reward_list(price, FinMe)
    Park_rw = reward_list(price, Park)
    FinGPT_rw = reward_list(price, FinGPT)
    PPO_rw = reward_list(price, PPO)
    A2C_rw = reward_list(price, A2C)
    DQN_rw = reward_list(price, DQN)
    # Perform Wilcoxon test
    print(f'Wilcoxon Test for {ticker} from {start} to {end}')
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, FinGPT_rw)
    print('Wilcoxon Test between FinMe and FinGPT - Statistic:', statistic, 'P-Value:', pvalue)
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, Park_rw)
    print('Wilcoxon Test between FinMe and Generative Agents - Statistic:', statistic, 'P-Value:', pvalue)
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, A2C_rw)
    print('Wilcoxon Test between FinMe and A2C - Statistic:', statistic, 'P-Value:', pvalue)
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, PPO_rw)
    print('Wilcoxon Test between FinMe and PPO - Statistic:', statistic, 'P-Value:', pvalue)
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, DQN_rw)
    print('Wilcoxon Test between FinMe and DQN - Statistic:', statistic, 'P-Value:', pvalue)

[*********************100%%**********************]  1 of 1 completed
Wilcoxon Test for TSLA from 2022-06-25 to 2023-04-25
Wilcoxon Test between FinMe and FinGPT - Statistic: 6862.0 P-Value: 6.117031045975193e-06
Wilcoxon Test between FinMe and Generative Agents - Statistic: 492.0 P-Value: 1.1134166479598554e-32
Wilcoxon Test between FinMe and A2C - Statistic: 1410.0 P-Value: 3.481933420152379e-27
Wilcoxon Test between FinMe and PPO - Statistic: 235.0 P-Value: 4.452151568661652e-34
Wilcoxon Test between FinMe and DQN - Statistic: 162.0 P-Value: 1.562599466637231e-34


/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_78194/585464115.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])
/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_78194/585464115.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])


In [14]:
if __name__ == "__main__":
    ticker = 'TSLA'
    start = '2022-10-06'
    end = '2023-04-10'

    # Fetch price data
    price = get_price(start, end, ticker)

    # Define file paths and columns for action data
    file_path_Me = '/Users/yuechenjiang/Desktop/CatMemo/result/Tsla-new-full.csv'
    file_path_Park = '/Users/yuechenjiang/Desktop/CatMemo/result/action_df_tsla_park_v2.csv'
    col_Me = ['date', 'direction']
    file_path_GPT = '/Users/yuechenjiang/Desktop/CatMemo/BenchMark/fingpt/tsla_curie.csv'
    col_GPT = ['dates', 'actions']
    file_path_A2C = '/Users/yuechenjiang/Desktop/CatMemo/result/TSLA_A2C_summary_data_seed2_full.csv'
    file_path_PPO = '/Users/yuechenjiang/Desktop/CatMemo/result/TSLA_PPO_summary_data_seed1_full.csv'
    file_path_DQN = '/Users/yuechenjiang/Desktop/CatMemo/result/TSLA_DQN_summary_data_seed1_full.csv'
    col_RL = ['date', 'action']

    # Fetch action data
    FinMe = get_action(start, end, ticker, file_path_Me, col_Me)
    Park = get_action(start, end, ticker, file_path_Park, col_Me)
    FinGPT = get_action(start, end, ticker, file_path_GPT, col_GPT)
    A2C = get_action(start, end, ticker, file_path_A2C, col_RL)
    PPO = get_action(start, end, ticker, file_path_PPO, col_RL)
    DQN = get_action(start, end, ticker, file_path_DQN, col_RL)
    
    # Calculate rewards
    FinMe_rw = reward_list(price, FinMe)
    Park_rw = reward_list(price, Park)
    FinGPT_rw = reward_list(price, FinGPT)
    PPO_rw = reward_list(price, PPO)
    A2C_rw = reward_list(price, A2C)
    DQN_rw = reward_list(price, DQN)
    # Perform Wilcoxon test
    print(f'Wilcoxon Test for {ticker} from {start} to {end}')
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, FinGPT_rw)
    print('Wilcoxon Test between FinMe and FinGPT - Statistic:', statistic, 'P-Value:', pvalue)
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, Park_rw)
    print('Wilcoxon Test between FinMe and Generative Agents - Statistic:', statistic, 'P-Value:', pvalue)
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, A2C_rw)
    print('Wilcoxon Test between FinMe and A2C - Statistic:', statistic, 'P-Value:', pvalue)
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, PPO_rw)
    print('Wilcoxon Test between FinMe and PPO - Statistic:', statistic, 'P-Value:', pvalue)
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, DQN_rw)
    print('Wilcoxon Test between FinMe and DQN - Statistic:', statistic, 'P-Value:', pvalue)

[*********************100%%**********************]  1 of 1 completed
Wilcoxon Test for TSLA from 2022-10-06 to 2023-04-10
Wilcoxon Test between FinMe and FinGPT - Statistic: 997.0 P-Value: 1.1779346203777291e-12
Wilcoxon Test between FinMe and Generative Agents - Statistic: 0.0 P-Value: 2.944866041483997e-22
Wilcoxon Test between FinMe and A2C - Statistic: 0.0 P-Value: 2.960958764325771e-22
Wilcoxon Test between FinMe and PPO - Statistic: 0.0 P-Value: 2.959463187045876e-22
Wilcoxon Test between FinMe and DQN - Statistic: 0.0 P-Value: 2.961172477329695e-22


/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_78194/585464115.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])
/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_78194/585464115.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])


In [15]:
if __name__ == "__main__":
    ticker = 'MSFT'
    start = '2022-10-06'
    end = '2023-04-10'

    # Fetch price data
    price = get_price(start, end, ticker)

    # Define file paths and columns for action data
    file_path_Me = '/Users/yuechenjiang/Desktop/CatMemo/result/Tsla-new-full.csv'
    file_path_Park = '/Users/yuechenjiang/Desktop/CatMemo/result/action_df_msft_v2_park.csv'
    col_Me = ['date', 'direction']
    file_path_GPT = '/Users/yuechenjiang/Desktop/CatMemo/BenchMark/fingpt/tsla_curie.csv'
    col_GPT = ['dates', 'actions']
    file_path_A2C = '/Users/yuechenjiang/Desktop/CatMemo/result/MSFT_A2C_summary_data_full.csv'
    file_path_PPO = '/Users/yuechenjiang/Desktop/CatMemo/result/MSFT_PPO_summary_data_2.csv'
    file_path_DQN = '/Users/yuechenjiang/Desktop/CatMemo/result/MSFT_DQN_summary_data.csv'
    col_RL = ['date', 'action']

    # Fetch action data
    FinMe = get_action(start, end, ticker, file_path_Me, col_Me)
    Park = get_action(start, end, ticker, file_path_Park, col_Me)
    FinGPT = get_action(start, end, ticker, file_path_GPT, col_GPT)
    A2C = get_action(start, end, ticker, file_path_A2C, col_RL)
    PPO = get_action(start, end, ticker, file_path_PPO, col_RL)
    DQN = get_action(start, end, ticker, file_path_DQN, col_RL)
    
    # Calculate rewards
    FinMe_rw = reward_list(price, FinMe)
    Park_rw = reward_list(price, Park)
    FinGPT_rw = reward_list(price, FinGPT)
    PPO_rw = reward_list(price, PPO)
    A2C_rw = reward_list(price, A2C)
    DQN_rw = reward_list(price, DQN)
    # Perform Wilcoxon test
    print(f'Wilcoxon Test for {ticker} from {start} to {end}')
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, FinGPT_rw)
    print('Wilcoxon Test between FinMe and FinGPT - Statistic:', statistic, 'P-Value:', pvalue)
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, Park_rw)
    print('Wilcoxon Test between FinMe and Generative Agents - Statistic:', statistic, 'P-Value:', pvalue)
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, A2C_rw)
    print('Wilcoxon Test between FinMe and A2C - Statistic:', statistic, 'P-Value:', pvalue)
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, PPO_rw)
    print('Wilcoxon Test between FinMe and PPO - Statistic:', statistic, 'P-Value:', pvalue)
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, DQN_rw)
    print('Wilcoxon Test between FinMe and DQN - Statistic:', statistic, 'P-Value:', pvalue)

[*********************100%%**********************]  1 of 1 completed
Wilcoxon Test for MSFT from 2022-10-06 to 2023-04-10
Wilcoxon Test between FinMe and FinGPT - Statistic: 152.0 P-Value: 2.4444303493712708e-20
Wilcoxon Test between FinMe and Generative Agents - Statistic: 0.0 P-Value: 2.9396619336549574e-22
Wilcoxon Test between FinMe and A2C - Statistic: 0.0 P-Value: 2.9590360122126185e-22
Wilcoxon Test between FinMe and PPO - Statistic: 2129.0 P-Value: 1.3366353811496614e-05
Wilcoxon Test between FinMe and DQN - Statistic: 2165.0 P-Value: 1.2575439242662996e-05


/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_78194/585464115.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])
/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_78194/585464115.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])


In [16]:
if __name__ == "__main__":
    ticker = 'NFLX'
    start = '2022-10-06'
    end = '2023-04-10'

    # Fetch price data
    price = get_price(start, end, ticker)

    # Define file paths and columns for action data
    file_path_Me = '/Users/yuechenjiang/Desktop/CatMemo/result/dat_df_nflx_new_v3.csv'
    file_path_Park = '/Users/yuechenjiang/Desktop/CatMemo/result/NFLX111.csv'
    col_Me = ['date', 'direction']
    file_path_GPT = '/Users/yuechenjiang/Desktop/CatMemo/BenchMark/fingpt/nflx_curie.csv'
    col_GPT = ['dates', 'actions']
    file_path_A2C = '/Users/yuechenjiang/Desktop/CatMemo/result/NFLX_A2C_summary_data_seed0.csv'
    file_path_PPO = '/Users/yuechenjiang/Desktop/CatMemo/result/NFLX_PPO_summary_data_seed2.csv'
    file_path_DQN = '/Users/yuechenjiang/Desktop/CatMemo/result/NFLX_DQN_summary_data_seed1.csv'
    col_RL = ['date', 'action']

    # Fetch action data
    FinMe = get_action(start, end, ticker, file_path_Me, col_Me)
    Park = get_action(start, end, ticker, file_path_Park, col_Me)
    FinGPT = get_action(start, end, ticker, file_path_GPT, col_GPT)
    A2C = get_action(start, end, ticker, file_path_A2C, col_RL)
    PPO = get_action(start, end, ticker, file_path_PPO, col_RL)
    DQN = get_action(start, end, ticker, file_path_DQN, col_RL)
    
    # Calculate rewards
    FinMe_rw = reward_list(price, FinMe)
    Park_rw = reward_list(price, Park)
    FinGPT_rw = reward_list(price, FinGPT)
    PPO_rw = reward_list(price, PPO)
    A2C_rw = reward_list(price, A2C)
    DQN_rw = reward_list(price, DQN)
    # Perform Wilcoxon test
    print(f'Wilcoxon Test for {ticker} from {start} to {end}')
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, FinGPT_rw)
    print('Wilcoxon Test between FinMe and FinGPT - Statistic:', statistic, 'P-Value:', pvalue)
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, Park_rw)
    print('Wilcoxon Test between FinMe and Generative Agents - Statistic:', statistic, 'P-Value:', pvalue)
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, A2C_rw)
    print('Wilcoxon Test between FinMe and A2C - Statistic:', statistic, 'P-Value:', pvalue)
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, PPO_rw)
    print('Wilcoxon Test between FinMe and PPO - Statistic:', statistic, 'P-Value:', pvalue)
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, DQN_rw)
    print('Wilcoxon Test between FinMe and DQN - Statistic:', statistic, 'P-Value:', pvalue)

[*********************100%%**********************]  1 of 1 completed
Wilcoxon Test for NFLX from 2022-10-06 to 2023-04-10
Wilcoxon Test between FinMe and FinGPT - Statistic: 15.0 P-Value: 4.2360608812866325e-22
Wilcoxon Test between FinMe and Generative Agents - Statistic: 7.0 P-Value: 5.120704583727878e-22
Wilcoxon Test between FinMe and A2C - Statistic: 0.0 P-Value: 2.938600954045116e-22
Wilcoxon Test between FinMe and PPO - Statistic: 0.0 P-Value: 2.9392374977738795e-22
Wilcoxon Test between FinMe and DQN - Statistic: 0.0 P-Value: 2.946567252968941e-22


/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_78194/585464115.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])
/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_78194/585464115.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])


In [17]:
if __name__ == "__main__":
    ticker = 'AMZN'
    start = '2022-10-06'
    end = '2023-04-10'

    # Fetch price data
    price = get_price(start, end, ticker)

    # Define file paths and columns for action data
    file_path_Me = '/Users/yuechenjiang/Desktop/CatMemo/result/dat_df_amzn_new_v5.csv'
    file_path_Park = '/Users/yuechenjiang/Desktop/CatMemo/result/AMZN111.csv'
    col_Me = ['date', 'direction']
    file_path_GPT = '/Users/yuechenjiang/Desktop/CatMemo/BenchMark/fingpt/amzn_curie-001.csv'
    col_GPT = ['dates', 'actions']
    file_path_A2C = '/Users/yuechenjiang/Desktop/CatMemo/result/AMZN_A2C_summary_data_seed0.csv'
    file_path_PPO = '/Users/yuechenjiang/Desktop/CatMemo/result/AMZN_PPO_summary_data_seed2_new.csv'
    file_path_DQN = '/Users/yuechenjiang/Desktop/CatMemo/result/AMZN_DQN_summary_data_seed2.csv'
    col_RL = ['date', 'action']

    # Fetch action data
    FinMe = get_action(start, end, ticker, file_path_Me, col_Me)
    Park = get_action(start, end, ticker, file_path_Park, col_Me)
    FinGPT = get_action(start, end, ticker, file_path_GPT, col_GPT)
    A2C = get_action(start, end, ticker, file_path_A2C, col_RL)
    PPO = get_action(start, end, ticker, file_path_PPO, col_RL)
    DQN = get_action(start, end, ticker, file_path_DQN, col_RL)
    
    # Calculate rewards
    FinMe_rw = reward_list(price, FinMe)
    Park_rw = reward_list(price, Park)
    FinGPT_rw = reward_list(price, FinGPT)
    PPO_rw = reward_list(price, PPO)
    A2C_rw = reward_list(price, A2C)
    DQN_rw = reward_list(price, DQN)
    # Perform Wilcoxon test
    print(f'Wilcoxon Test for {ticker} from {start} to {end}')
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, FinGPT_rw)
    print('Wilcoxon Test between FinMe and FinGPT - Statistic:', statistic, 'P-Value:', pvalue)
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, Park_rw)
    print('Wilcoxon Test between FinMe and Generative Agents - Statistic:', statistic, 'P-Value:', pvalue)
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, A2C_rw)
    print('Wilcoxon Test between FinMe and A2C - Statistic:', statistic, 'P-Value:', pvalue)
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, PPO_rw)
    print('Wilcoxon Test between FinMe and PPO - Statistic:', statistic, 'P-Value:', pvalue)
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, DQN_rw)
    print('Wilcoxon Test between FinMe and DQN - Statistic:', statistic, 'P-Value:', pvalue)

[*********************100%%**********************]  1 of 1 completed
Wilcoxon Test for AMZN from 2022-10-06 to 2023-04-10
Wilcoxon Test between FinMe and FinGPT - Statistic: 0.0 P-Value: 2.9601040600868202e-22
Wilcoxon Test between FinMe and Generative Agents - Statistic: 6.0 P-Value: 3.422351004243837e-22
Wilcoxon Test between FinMe and A2C - Statistic: 51.0 P-Value: 1.4830444235968182e-21
Wilcoxon Test between FinMe and PPO - Statistic: 0.0 P-Value: 2.9558340835819847e-22
Wilcoxon Test between FinMe and DQN - Statistic: 0.0 P-Value: 2.9576481019816267e-22


/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_78194/585464115.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])


In [18]:
if __name__ == "__main__":
    ticker = 'COIN'
    start = '2022-10-06'
    end = '2023-04-10'

    # Fetch price data
    price = get_price(start, end, ticker)

    # Define file paths and columns for action data
    file_path_Me = '/Users/yuechenjiang/Desktop/CatMemo/result/COIN20221004-20230802.csv'
    file_path_Park = '/Users/yuechenjiang/Desktop/CatMemo/result/action_df_coin_park_v2.csv'
    col_Me = ['date', 'direction']
    file_path_GPT = '/Users/yuechenjiang/Desktop/CatMemo/BenchMark/fingpt/coin_curie.csv'
    col_GPT = ['dates', 'actions']

    # Fetch action data
    FinMe = get_action(start, end, ticker, file_path_Me, col_Me)
    Park = get_action(start, end, ticker, file_path_Park, col_Me)
    FinGPT = get_action(start, end, ticker, file_path_GPT, col_GPT)

    # Calculate rewards
    FinMe_rw = reward_list(price, FinMe)
    FinGPT_rw = reward_list(price, FinGPT)
    Park_rw = reward_list(price, Park)
    # Perform Wilcoxon test
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, FinGPT_rw)
    print('Wilcoxon Test between FinMe and FinGPT - Statistic:', statistic, 'P-Value:', pvalue)
    statistic, pvalue = perform_wilcoxon_test(FinMe_rw, Park_rw)
    print('Wilcoxon Test between FinMe and Generative Agents - Statistic:', statistic, 'P-Value:', pvalue)

[*********************100%%**********************]  1 of 1 completed
Wilcoxon Test between FinMe and FinGPT - Statistic: 186.0 P-Value: 2.3802627822203768e-20
Wilcoxon Test between FinMe and Generative Agents - Statistic: 1831.0 P-Value: 3.440953727541258e-07


/var/folders/kg/pwzs55h51hb6x5vbsqjvxhjc0000gn/T/ipykernel_78194/585464115.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col[0]] = pd.to_datetime(df[col[0]])
